In [12]:
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import torch
import torch.nn as nn
import torchvision.transforms as transforms

from concurrent.futures import ThreadPoolExecutor

import rasterio as rio
from rasterio.crs import CRS

import cv2
from affine import Affine
import numpy as np
import glob

from tqdm.notebook import tqdm

from affine import Affine

from WorldFileUtils import *

Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
def prep(image):
    image = np.asarray(image).astype(np.uint8)
    image = np.uint8(255 - image)
    return image

def prepedges(image):
    image = np.asarray(image).astype(np.uint8)
    image = cv2.Canny(image,50,100)
    image = cv2.dilate(image, np.ones((3,3), np.uint8), iterations=30)
    return np.asarray(image ).astype(np.uint8)

# REFERENCE IMAGE FROM QGIS
template = prep(Image.open(r"Harris_Boundary_hollow.png"))[:,:,0]

# TILE INDEX TO TEST ON
test_preprocess = prepedges(Image.open(r"masks_black.png"))

cv2.imwrite("test.png", test_preprocess)
cv2.imwrite("template.png", template)

True

In [3]:
class ReturnValues:
    def __init__(self, _result, _mask, _rotated_template, _scale, _angle):
        self.result = _result
        self.mask = _mask
        self.rotated_template = _rotated_template
        self.scale = _scale
        self.angle = _angle

def wrapPattern(scale):
    return patternMatch(test_preprocess, template, scale, 0)

def patternMatch(test_preprocess, template, scale, angle, method=cv2.TM_SQDIFF):
    # Resize the template based on the current scale
    scaled_template = cv2.resize(template, None, fx=scale, fy=scale)
    
    # Rotate the template based on the current angle
    rotation_matrix = cv2.getRotationMatrix2D((scaled_template.shape[1] / 2, scaled_template.shape[0] / 2), angle, 1.0)
    rotated_template = cv2.warpAffine(scaled_template, rotation_matrix, (scaled_template.shape[1], scaled_template.shape[0]))
        
    mask = np.where(rotated_template == 0, 0, 255).astype(np.uint8)
    # Perform template matching
    
    
    result = cv2.matchTemplate(test_preprocess, rotated_template, method, mask=mask)
    return ReturnValues(result, mask,rotated_template, scale, angle)

def postprocess_results(result_list, scales, opt_max):
    elem_list = [x.flatten() for x in result_list]
    elem_list = np.hstack(elem_list)

    if opt_max:
        thresh = np.percentile(elem_list, 99.9)
        loc_list = [x > thresh for x in result_list]
    else:
        thresh = np.percentile(elem_list, 0.1)
        loc_list = [x < thresh for x in result_list]

    votes = np.array([np.count_nonzero(x) for x in loc_list])
    rescale_factor = np.sum(votes * scales) / np.sum(votes)

    x_list = np.hstack([np.where(x)[0] for x in loc_list])
    y_list = np.hstack([np.where(x)[1] for x in loc_list])

    x = int(np.median(x_list))
    y = int(np.median(y_list))

    br = (int(y + template.shape[1] * rescale_factor), int(x + template.shape[0] * rescale_factor))
    
    return x, y, br, rescale_factor

In [4]:
def wrapSearch(initial_guess, perturbance, retall=True):
    scales = np.arange(initial_guess-perturbance, initial_guess+perturbance+1e-5, perturbance/2)

    # Initialize a list to store the matches
    all_matches = []

    # Initialize variables to store the best match
    best_match_scale = 1.0
    best_match_angle = 0.0

    opt_max = False

    if opt_max:
        best_match_value = -1 * np.inf
    else:
        best_match_value = np.inf

    result_list = list()
    best_loc_list = list()
    rect_corner_list = list()

    with ThreadPoolExecutor(max_workers=4) as executor:
        for retvalue in tqdm(executor.map(wrapPattern, scales), total=len(scales)):

            mask = retvalue.mask
            result = retvalue.result
            rotated_template = retvalue.rotated_template
            scale = retvalue.scale
            angle = retvalue.angle
            if not opt_max:
                result = np.sqrt(result / np.count_nonzero(mask))

            result_list.append(result)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

            if opt_max:
                best_val = max_val
                best_loc = max_loc
                bottom_right = (best_loc[0] + int(rotated_template.shape[1]), best_loc[1] + int(rotated_template.shape[0]))

                best_loc_list.append(best_loc)
                rect_corner_list.append(bottom_right)

            else:
                best_val = min_val
                best_loc = min_loc
                bottom_right = (best_loc[0] + int(rotated_template.shape[1]), best_loc[1] + int(rotated_template.shape[0]))
                best_loc_list.append(best_loc)
                rect_corner_list.append(bottom_right)            

            print(scale, angle, best_val)
        
        postprocess = postprocess_results(result_list, scales, opt_max)
    
    if not retall:
        return postprocess[3]
    return postprocess

In [5]:
curr_guess = 0.8
perturbations = [0.1, 0.05, 0.025, 0.0125]

for i, pert in enumerate(perturbations):
    x, y, br, rescale_factor = wrapSearch(curr_guess, pert)
    curr_guess = rescale_factor
    print(f"New Guess: {curr_guess}")

  0%|          | 0/5 [00:00<?, ?it/s]

0.7000000000000001 0 216.62223399738718
0.7500000000000001 0 215.6942329081225
0.8000000000000002 0 216.78464493077408
0.8500000000000002 0 204.41115459919004
0.9000000000000002 0 208.84378380878118
New Guess: 0.868405879365459


  0%|          | 0/5 [00:00<?, ?it/s]

0.8184058793654589 0 215.9508292292257
0.843405879365459 0 206.19018216747364
0.868405879365459 0 180.93880574374046
0.893405879365459 0 206.6386741173798
0.918405879365459 0 212.63262653488525
New Guess: 0.868405879365459


  0%|          | 0/5 [00:00<?, ?it/s]

0.843405879365459 0 206.19018216747364
0.8559058793654589 0 203.9508696244984
0.8684058793654589 0 180.93880574374046
0.8809058793654588 0 190.5873356924205
0.8934058793654588 0 206.6386741173798
New Guess: 0.8705303580813294


  0%|          | 0/5 [00:00<?, ?it/s]

0.8580303580813294 0 201.06806545887088
0.8642803580813294 0 183.98855551085148
0.8705303580813294 0 180.74920454884904
0.8767803580813294 0 186.84215725664637
0.8830303580813293 0 194.825116455651
New Guess: 0.8698789292739293


In [6]:
rescale_factor

0.8698789292739293

In [7]:
three_band = np.dstack([test_preprocess,test_preprocess,test_preprocess])
    
# _ = cv2.rectangle(three_band, best_match_loc, best_match_br, (0, 0, 255), 5)
_ = cv2.rectangle(three_band, (y, x), br, (0, 255, 0), 5)

In [8]:
cv2.imwrite('testout.png', three_band)

True

In [33]:
tfw_file        = "Harris_Boundary_hollow.pgw"
template_affine = get_affine_from_geotransform(get_geotransform_from_tfw(tfw_file)[0])
calc_affine     = Affine(1 / rescale_factor, 0, -y, 0, 1 / rescale_factor, -x)

def write_world_file(affine, filename):
    with open(filename, 'w') as file:
        file.write(f'{affine.a}\n')
        file.write(f'{affine.d}\n')
        file.write(f'{affine.b}\n')
        file.write(f'{affine.e}\n')
        file.write(f'{affine.c}\n')
        file.write(f'{affine.f}\n')

In [34]:
# out_affine = combineAffine(template_affine, calc_affine)
out_affine = template_affine * calc_affine
write_world_file(out_affine, "data/TileIndices/48201CIND0_0992.tfw")